In [24]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.messages import HumanMessage
from langgraph.types import Command

In [25]:
def read_email_tool(email_id:str)->str:
    """Mock funtion to read an email by it's ID"""
    return f"Email content for ID   :{email_id}"

def send_email_tool(recipient:str,  subject:str,body:str)->str:
    """Mock function to send an email"""
    return f"Email sent to {recipient} with subject '{subject}'"


In [26]:
model=ChatOllama(model="llama3.1:8b")

In [27]:
agent=create_agent(model=model,checkpointer=InMemorySaver(),tools=[read_email_tool,send_email_tool],
                   middleware=[HumanInTheLoopMiddleware(interrupt_on={
                       "send_email_tool":{
                           "allowed_decisions":["approve","edit","reject"]
                       },
                       "read_email_tool":False
                   })]
                   )

In [28]:
config={"configurable":{"thread_id":"test-approve"}}
result=agent.invoke(
    {"messages":[HumanMessage(content="Send mail to john@test.com with subject 'Hello' and body 'How are you?'")]},
    config=config
)

In [29]:
result

{'messages': [HumanMessage(content="Send mail to john@test.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='22464af6-744b-48f2-a863-e96e474ea879'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-02-16T11:58:20.633127753Z', 'done': True, 'done_reason': 'stop', 'total_duration': 703451326, 'load_duration': 103519781, 'prompt_eval_count': 219, 'prompt_eval_duration': 19198758, 'eval_count': 34, 'eval_duration': 552773371, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019c6650-d718-7081-92a3-d413bc34518b-0', tool_calls=[{'name': 'send_email_tool', 'args': {'body': 'How are you?', 'subject': 'Hello', 'recipient': 'john@test.com'}, 'id': '99e49bca-4bb6-456c-a114-755aba8364df', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 219, 'output_tokens': 34, 'total_tokens': 253})],
 '__interrupt__': [Interrupt(value={'a

In [30]:
if "__interrupt__" in result:
    print("Paused ! Approving... ")
    result=agent.invoke(
        Command(
            resume={
                "decisions":[
                    {"type":"approve"}
                ]
            }
        ),
        config=config
    )
    print(f"Result: {result['messages'][-1].content}")

Paused ! Approving... 
Result: The email has been successfully sent.


In [31]:
result

{'messages': [HumanMessage(content="Send mail to john@test.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='22464af6-744b-48f2-a863-e96e474ea879'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-02-16T11:58:20.633127753Z', 'done': True, 'done_reason': 'stop', 'total_duration': 703451326, 'load_duration': 103519781, 'prompt_eval_count': 219, 'prompt_eval_duration': 19198758, 'eval_count': 34, 'eval_duration': 552773371, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019c6650-d718-7081-92a3-d413bc34518b-0', tool_calls=[{'name': 'send_email_tool', 'args': {'body': 'How are you?', 'subject': 'Hello', 'recipient': 'john@test.com'}, 'id': '99e49bca-4bb6-456c-a114-755aba8364df', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 219, 'output_tokens': 34, 'total_tokens': 253}),
  ToolMessage(content="Email sent to jo